In [1]:
from tensorflow.keras.models import load_model

In [2]:
#!pip install imutils

In [3]:
from imutils.contours import sort_contours

In [4]:
import numpy as np
import argparse
import imutils
import cv2

In [5]:
# construct the argument parser and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-i", "--image", required=True,help="path to input image")
# ap.add_argument("-m", "--model", type=str, required=True,help="path to trained handwriting recognition model")
# args = vars(ap.parse_args())

# load the handwriting OCR model
print("[INFO] loading handwriting OCR model...")
#model = load_model(args["model"])
model = load_model("handwriting.model")

# load the input image from disk, convert it to grayscale, and blur
# it to reduce noise
image = cv2.imread('images/number.jpeg')
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
blurred = cv2.GaussianBlur(gray, (5, 5), 0)


[INFO] loading handwriting OCR model...


In [6]:
# perform edge detection, find contours in the edge map, and sort the
# resulting contours from left-to-right
edged = cv2.Canny(blurred, 30, 150)
cnts = cv2.findContours(edged.copy(), cv2.RETR_EXTERNAL,
    cv2.CHAIN_APPROX_SIMPLE)
cnts = imutils.grab_contours(cnts)
cnts = sort_contours(cnts, method="left-to-right")[0]

# initialize the list of contour bounding boxes and associated
# characters that we'll be OCR'ing
chars = []


In [7]:
# loop over the contours
for c in cnts:
    # compute the bounding box of the contour
    (x, y, w, h) = cv2.boundingRect(c)

    # filter out bounding boxes, ensuring they are neither too small
    # nor too large
    if (w >= 5 and w <= 150) and (h >= 15 and h <= 120):
        # extract the character and threshold it to make the character
        # appear as *white* (foreground) on a *black* background, then
        # grab the width and height of the thresholded image
        roi = gray[y:y + h, x:x + w]
        thresh = cv2.threshold(roi, 0, 255,
            cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)[1]
        (tH, tW) = thresh.shape

        # if the width is greater than the height, resize along the
        # width dimension
        if tW > tH:
            thresh = imutils.resize(thresh, width=32)

        # otherwise, resize along the height
        else:
            thresh = imutils.resize(thresh, height=32)

        # re-grab the image dimensions (now that its been resized)
        # and then determine how much we need to pad the width and
        # height such that our image will be 32x32
        (tH, tW) = thresh.shape
        dX = int(max(0, 32 - tW) / 2.0)
        dY = int(max(0, 32 - tH) / 2.0)

        # pad the image and force 32x32 dimensions
        padded = cv2.copyMakeBorder(thresh, top=dY, bottom=dY,
            left=dX, right=dX, borderType=cv2.BORDER_CONSTANT,
            value=(0, 0, 0))
        padded = cv2.resize(padded, (32, 32))

        # prepare the padded image for classification via our
        # handwriting OCR model
        padded = padded.astype("float32") / 255.0
        padded = np.expand_dims(padded, axis=-1)

        # update our list of characters that will be OCR'd
        chars.append((padded, (x, y, w, h)))


In [8]:

# extract the bounding box locations and padded characters
boxes = [b[1] for b in chars]
chars = np.array([c[0] for c in chars], dtype="float32")

# OCR the characters using our handwriting recognition model
preds = model.predict(chars)

# define the list of label names
labelNames = "0123456789"
labelNames += "ABCDEFGHIJKLMNOPQRSTUVWXYZ"
labelNames = [l for l in labelNames]

s=''
# loop over the predictions and bounding box locations together
for (pred, (x, y, w, h)) in zip(preds, boxes):
    # find the index of the label with the largest corresponding
    # probability, then extract the probability and label
    i = np.argmax(pred)
    prob = pred[i]
    label = labelNames[i]
    s=s+label

    # draw the prediction on the image
    print("[INFO] {} - {:.2f}%".format(label, prob * 100))
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
    cv2.putText(image, label, (x - 10, y - 10),
        cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 255, 0), 2)

    # show the image
print(s)
s=s.lower()
cv2.imshow("Image", image)
cv2.waitKey(0)


[INFO] 7 - 36.24%
[INFO] 2 - 65.86%
[INFO] 5 - 48.25%
[INFO] 2 - 96.88%
[INFO] 7 - 90.07%
[INFO] 3 - 88.43%
[INFO] E - 62.08%
[INFO] C - 39.71%
[INFO] 4 - 96.50%
[INFO] 3 - 56.03%
[INFO] 7 - 75.03%
[INFO] 4 - 90.01%
[INFO] L - 20.58%
[INFO] 2 - 90.72%
[INFO] 1 - 98.03%
[INFO] 2 - 90.81%
[INFO] T - 86.33%
[INFO] 7 - 51.90%
[INFO] 4 - 55.81%
[INFO] 3 - 95.71%
[INFO] 9 - 33.68%
[INFO] 7 - 97.44%
[INFO] 2 - 22.58%
[INFO] V - 43.82%
[INFO] 5 - 91.29%
[INFO] 3 - 99.37%
[INFO] 3 - 52.24%
[INFO] 2 - 82.36%
[INFO] 1 - 90.95%
[INFO] 6 - 48.60%
[INFO] Q - 34.33%
[INFO] 7 - 95.39%
[INFO] 7 - 72.79%
[INFO] R - 24.47%
725273EC4374L212T743972V533216Q77R


-1

In [9]:
#!pip install autocorrect

In [10]:
# from autocorrect import Speller

# spell = Speller(lang='en')

# print(spell(s))

In [11]:
#!pip install textblob

In [12]:
from textblob import TextBlob        # incorrect spelling
print("original text: "+str(s))
 
b = TextBlob(s)
 
# prints the corrected spelling
print("corrected text: "+str(b.correct()))

original text: 725273ec4374l212t743972v533216q77r
corrected text: 725273ec4374l212t743972v533216q77r
